# Business Understanding

## Wie (de opdrachtgever)? 

Appbakkers (Corné & Guido) is een bedrijf die (mobiele) applicaties ontwikkeld.  

 

## Wat (de opdracht)? 

Bij opname van een patiënt wordt er een elektrocardiogram (ECG) gemaakt. Hierbij worden alle elektrische impulsen in het hart geregistreerd, waardoor kan worden afgeleid welke gebieden van het hart naar behoren functioneren.  

Arts-onderzoekers vragen zich af of het aantal hartslagen per minuut een goede voorspeller kan zijn van de calciumscore. Gebruik de ECG-datasets om per dataset het gemiddelde aantal hartslagen per minuut te bepalen (bedenk goed welke onderdelen van de ECGs je hiervoor allemaal gaat gebruiken): indien deze boven de 100 slagen per minuut is heet dit tachycardie, indien deze onder de 60 slagen per minuut is heet dit bradycardie.  

Onderzoek daarna met deze resultaten en de calciumscore uitkomsten of de aanwezigheid van bradycardie/tachycardie/normale hartslag (en meer specifiek het aantal hartslagen per minuut) de calciumscore kan voorspellen. Bepaal hierna of de datasets een regelmatig of onregelmatig hartritme vertonen en probeer je model hiermee te verbeteren. Kan je model nu accuraat de calciumscore voorspellen? Zijn er risico’s bij het gebruik van je model in de praktijk? Als je nog tijd over hebt, is het interessant om je gegevens te combineren met die van opdracht 1. Kan je elkaars modellen verder verbeteren? 

 

## Wanneer (tijdsframe)? 

Deze opdracht voeren we uit vanaf 13 april ’22. Op deze dag hebben we de kennismaking gehad met onze opdrachtgever. Hierin hebben we de opdracht besproken en een aantal afspraken gemaakt. De einddatum van de opdracht zal zijn op 3 juni ’22. 

 

## Waarom (doel)? 

Het doel is om te achterhalen of hartslag een goede voorspeller kan zijn van de calciumscore. 

Als het model gemaakt is om dit te voorspellen kan deze verbeterd/uitgebreid worden met de resultaten en de calciumscore uitkomsten of de aanwezigheid van bradycardie/tachycardie/normale hartslag (en meer specifiek het aantal hartslagen per minuut) de calciumscore kan voorspellen.  

 

## Wat (wat gaan we gebruiken)? 

Door middel van Jupyter notebooks gaan wij het model ontwikkelen.  
ECG viewer om ECG´s bestanden door te lezen. 

# Data Understanding

Wij hebben 2 publieke datasets gekregen om mee bezig te gaan. Deze zijn "cardio_train.csv" en "riskchartsampledata.xlsx". Hieronder is te lezen wat de kolommen in deze datasets betekenen.

### "cardio_train.csv"

* id          
    De id van de rij, **integer** <br/>
* age         
    De leeftijd van de patient in dagen, **integer** <br/>
* gender      
    Het geslacht van de patient, waarbij 1 een man is en 2 een vrouw, **category** <br/>
* height      
    De lengte van de patient, gegeven in centimeters, **integer** <br/>
* weight      
    Het gewicht van de patient, gegeven in kilogram, **float** <br/>
* ap_hi       
    De bovendruk van de patient, **integer**  <br/>
* ap_lo       
    De onderdruk van de patient, **integer**  <br/>
* cholesterol <br/>
    Hoe veel cholesterol de patient heeft, **category**  <br/>
* gluc        
    Hoe veel glucoses de patient heeft, **category**  <br/>
* smoke       
    Of de patient rookt, **boolean** <br/>
* alco        
    Of de patient alcohol drinkt, **boolean** <br/>
* active      
    Of de patient actief is in het dagelijks leven, **boolean** <br/>
* cardio      
    Of de patient een cardiovasculaire ziekte heeft, **boolean** <br/>

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [86]:
cardio_train = pd.read_csv("Data/cardio_train.csv", delimiter=';')
#riskchart = pd.read_excel("Data/riskchartsampledata.xlsx", true_values=['Yes','Diabetes','Smoker'], false_values=['No','Non-diabetes','Non-smoker'])

In [88]:
profile = pp.ProfileReport(cardio_train)
profile.to_file("Cardio_train_Report.html")
#GiveInformationOfDataframe(cardio_train)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [105]:
sns.pairplot(cardio_train)

In [94]:
cardio_train[['gender','height']].groupby('gender').agg(np.mean)

,height
gender,
1,161.355612
2,169.947895


De personen met 2 als gender zijn gemiddeld langer, dus is de 2 een man en 1 een vrouw.

In [116]:
# Functie om uit te rekenen hoeveel procent een cardiovasuclaire ziekte heeft bij een andere conditie.
def percentage(waardeInKolom, tabelKolom):
    percentage = cardio_train[(cardio_train.cardio == 1) & (tabelKolom == waardeInKolom)]['id'].count() / 70000 * 100
    percentage = round(percentage, 2)
    print (percentage,'%')

In [118]:
percentage(2, cardio_train.gender)

17.66 %


### Uitleg van de data

De dataset ‘cardio_train.csv’ bevat 70000 rijen en 13 kolommen. De inhoud van de kolommen wordt hieronder uitgelegd. Daarnaast worden er een aantal statistieken meegegeven.

Id: in de kolom ‘id’ staan de id’s van elke rij. Genummerd vanaf 0 tot en met 69999.

Age: in de kolom ‘age’ staat de leeftijd van de patiënt in dagen.  De minimale leeftijd is 10798 dagen  wat omgerekend naar jaren 29 is. De hoogste leeftijd is 23713 dagen, dit is omgerekend 64 jaar. De gemiddelde leeftijd is 53 jaar.

Gender: De kolom ‘gender’ bevat het geslacht van de patiënt, een ‘1’ staat voor een vrouw en een ‘2’ staat voor een man. 65% van de patiënten zijn vrouw. Uit de data blijkt dat 32,31% van de vrouwen een cardiovasculaire ziekte heeft en 17,66% van de mannen.

Height: in de kolom ‘height’ staat de lengte van de patiënt in centimeters. De kleinste persoon is volgens de data 55 centimeter en de langste persoon is volgens de data 250 centimeter. De gemiddelde lengte is 164,36 centimeter.

Weight: De kolom ‘weight’ bevat het gewicht van de patiënten in kilogrammen. De lichtste persoon in volgens de data 10 kilogram en de zwaarste persoon is 200 kilogram. Het gemiddelde gewicht is 74,2 kilo.

Ap_hi: De kolom ‘ap_hi’  bevat de bovendruk van de patiënten in mmHg. De hoogste waarde is 16020 en de laagste waarde is -150. Het gemiddelde ligt op 128.

Ap_lo: De kolom ‘ap_lo’  bevat de onderdruk van de patiënten in mmHg. De hoogste waarde is 11000 en de laagste waarde is -70. Het gemiddelde ligt op 96.

Colesterol: de kolom ‘colesterol’ bevat het cholesterol gehalte van de patiënt. De kolom bevat drie waarden ‘1’ normaal, ‘2’ boven normaal en ‘3’ ruim boven normaal. De meesten vallen in categorie 1. 

Gluc: de kolom ‘gluc’ bevat het glucose gehalte van de patiënt. De kolom bevat drie waarden ‘1’ normaal, ‘2’ boven normaal en ‘3’ ruim boven normaal. De meesten vallen in categorie 1. 

Smoke: de kolom ‘smoke’ geeft aan of de patiënt rookt. De kolom bevat twee waarden, ‘0’ de patiënt rookt niet en ‘1’ de patiënt rookt. Uit de data blijkt dat 4,18% van de patiënten die roken een cardiovasculaire ziekte heeft en 45,79% van de patiënten die roken een cardiovasculaire ziekte heeft.

Alco: de kolom ‘alco’ geeft aan of de patiënt alcohol drinkt. De kolom bevat twee waarden, ‘0’ de patiënt drinkt geen alcohol en ‘1’ de patiënt drinkt alcohol. Uit de data blijkt dat 2,60% van de patiënten die drinken een cardiovasculaire ziekte heeft en 47,37% van de patiënten die niet drinken een cardiovasculaire ziekte heeft.

Active: de kolom ‘active’ geeft aan of de patiënt actief is. De kolom bevat twee waarden, ‘0’ de patiënt is niet actief en ‘1’ de patiënt is actief. Uit de data blijkt dat 39,45% van de patiënten die actief zijn een cardiovasculaire ziekte heeft en 10,51% van de patiënten die niet actief zijn een cardiovasculaire ziekte heeft.

Cardio: de kolom ‘cardio’ geeft aan of de patiënt een cardiovasculaire ziekte heeft. De kolom bevat twee waarden, ‘0’ de patiënt heeft geen cardiovasculaire ziekte en ‘1’ de patiënt heeft een cardiovasculaire ziekte. In de data heeft 49,97% van de patiënten een cardiovasculaire ziekte.

# Data Preparation

In [95]:
cardio_train.astype({'gender':'category','cholesterol':'category','gluc':'category','smoke':'bool','alco':'bool','active':'bool','cardio':'bool'}, copy=False)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,False,False,True,False
1,1,20228,1,156,85.0,140,90,3,1,False,False,True,True
2,2,18857,1,165,64.0,130,70,3,1,False,False,False,True
3,3,17623,2,169,82.0,150,100,1,1,False,False,True,True
4,4,17474,1,156,56.0,100,60,1,1,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,True,False,True,False
69996,99995,22601,1,158,126.0,140,90,2,2,False,False,True,True
69997,99996,19066,2,183,105.0,180,90,3,1,False,True,False,True
69998,99998,22431,1,163,72.0,135,80,1,2,False,False,False,True


### Aanpassen van data

In [96]:
cardio_train.astype('category',copy=False)

cardio_train['age_in_years'] = (cardio_train['age']/365).agg(np.round)

#cardio_train['age_category'] = pd.cut(cardio_train['age_in_years'], range(5,105,10))
#age_cat = [str(x.left) + '-' + str(x.right-1) for x in cardio_train['age_category']]
#cardio_train['age_category'] = age_cat
#cardio_train['age_category'].astype('category',copy=False)

cardio_train['bmi'] = cardio_train['weight'] / (cardio_train['height']/100)**2 

cardio_train['ap_difference'] = cardio_train['ap_hi'] - cardio_train['ap_lo']



In [97]:
cardio_train.drop_duplicates(inplace=True)
data = cardio_train

# ap_hi kan niet lager zijn dan ap_lo
data = data[:][data['ap_hi'] > data['ap_lo']]

data = data[~(data['bmi']>60)]


high_ap_filter = ((data["ap_hi"]>220) | (data["ap_lo"]>180))
data = data[~high_ap_filter]

low_ap_filter = ((data["ap_hi"] < 40) | (data["ap_lo"] < 40))
data = data[~low_ap_filter]

labels = data['cardio']

# Use this as a filter for which columns you dont want in your model
columns_to_remove = ['id','bmi','gluc', 'ap_lo', 'smoke', 'active']
for column in columns_to_remove:
    data = data.loc[:, data.columns != column]